# Extract Bounding Box Images from Video

This example takes as an input a rrd with a video asset and 2D bounding boxes
and extracts the cropped images for the associated frame.

In [ ]:
import rerun as rr
import rerun.utilities.datafusion.functions as rr_dfn
import pyarrow as pa
from datafusion import SessionContext, col

In [ ]:
# In this example, you will need a copy of the rrd from the Detect & Track Example.
# Update the file path below accordingly.

original_recording = rr.dataframe.load_recording("detect_and_track_example.rrd")

In [ ]:
# In order to turn this into a DataFusion DataFrame, we need to create a view
# that includes our `latest_at` specification. Otherwise the data for the Pinhole
# and the data for the DepthImage will be misaligned.

original_recording_view = (
    original_recording
    .view(index="log_time", contents="/**", include_indicator_columns=True)
)

In [ ]:
# Set some constants that will be reused below

FRAME_INDEX = "frame"
INPUT_ENTITY_PATH = "/video/tracked/14"
INPUT_VIDEO_PATH = "/video"
OUTPUT_ENTITY_PATH = "/cropped_images"

In [ ]:
# Filter the selection to retrieve only the required data

required_input_columns = [
    "frame",
    "log_tick",
    "log_time",
    f"{INPUT_ENTITY_PATH}:Position2D",
    f"{INPUT_ENTITY_PATH}:HalfSize2D",
    f"{INPUT_VIDEO_PATH}:Blob",
    f"{INPUT_VIDEO_PATH}:MediaType",
]

In [ ]:
# Create the DataFusion context and DataFrame from record batches provided
# by the view above.

batches = [r for r in original_recording_view.select(*required_input_columns)]
ctx = SessionContext()
df = ctx.create_dataframe([batches])


In [ ]:
# Call the Rerun function to extract the images from the video

df_images = rr_dfn.extract_bounding_box_images_from_video(
    df,
    FRAME_INDEX,
    INPUT_ENTITY_PATH,
    INPUT_VIDEO_PATH,
    OUTPUT_ENTITY_PATH
)

In [ ]:
# To visualize the results, create a recording stream

local_rec = rr.RecordingStream("image_extraction")
local_rec.spawn()

In [ ]:
# The DataFrame above will contain the original data as well as the generated images.
# We select down to only what we want to send to the viewer.

df_images = df_images.select(
    "frame",
    "log_tick",
    "log_time",
    f"{OUTPUT_ENTITY_PATH}:ImageBuffer",
    f"{OUTPUT_ENTITY_PATH}:ImageFormat",
)

In [ ]:
# Send the original data so we can show the new images alongside the original data

local_rec.send_recording(original_recording)

In [ ]:
# Convert the DataFusion DataFrame into a pyarrow Table and send it to the viewer

table_result = pa.table(df_images)
rr.dataframe.send_dataframe(table_result, rec=local_rec)